In [1]:
import vector_database
import embedding
import wikipedia

In [2]:
db = vector_database.VectorDatabaseWraper("wiki_db")


In [3]:
query = "Tell me about some famous people"
emb = embedding.embedding([query])[0]
search_results, likeness = db.search(emb, 4)

In [4]:
documents = []
for result in search_results:
    section_text = wikipedia.get_section_text(result['page_title'], result['section_title'])
    documents.append(section_text)

In [5]:
documents

['"Tennis balles" are mentioned by William Shakespeare in his play Henry V (1599), when a basket of them is given to King Henry as a mockery of his youth and playfulness.\nDavid Foster Wallace, an amateur tennis player himself at Urbana High School in Illinois, included tennis in many of his works of non-fiction and fiction including "Tennis Player Michael Joyce\'s Professional Artistry as a Paradigm of Certain Stuff about Choice, Freedom, Discipline, Joy, Grotesquerie, and Human Completeness", the autobiographical piece "Derivative Sport in Tornado Alley", and Infinite Jest, which is partially set at the fictional "Enfield Tennis Academy" in Massachusetts.\nJapanese manga series The Prince of Tennis revolves around the tennis prodigy Echizen Ryoma and tennis matches between rival schools.\nThe Royal Tenenbaums (2001) is a film that features Richie Tenenbaum (Luke Wilson), a tennis pro who suffers from depression and has a breakdown on court in front of thousands of fans.\nWimbledon (2

In [6]:
for document in documents:
    print(len(document.split(" ")))

342
652
357
79


In [17]:
import os
import ollama
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import TransformChain


llmModel = 'llama3.1:8b'
model = ChatOllama(model = llmModel)
llmModelsmall = 'llama3.2:1b'
model_small = ChatOllama(model = llmModelsmall)

# generate different versions of provided question to retrieve context based on
# different perspectives
# query_prompt = PromptTemplate(input_variables=["question"], template = "")

template_RAG = "Try to answer the question based on YOUR knowledge OR the context. Context: {context} \n Question: {question}"
template_summarize = "Remove UNRELEVANT information from the following CONTEXT while taking into account the QUESTION. Output ONLY summarized content. Context: {context} \n Question: {question}"
summarize = ChatPromptTemplate.from_template(template_summarize)
prompt = ChatPromptTemplate.from_template(template_RAG)

#print(fake_retriever())


def fake_retriever(input_: dict) -> str:
    # constext
    file_name = "story1.txt"
    file_path = "../docs/" + file_name
    f = open(file_path, "r")
    return f.read()
    #return ""

def empty_retriever(input_: dict) -> str:
    print(input_)
    return "use only your built in knowledge"

def wikipedia_retriever(input_: dict) -> str:
    emb = embedding.embedding([input_])[0]
    search_results, likeness = db.search(emb, 4)
    documents = []
    for result in search_results:
        section_text = wikipedia.get_section_text(result['page_title'], result['section_title'])
        documents.append(section_text)
    return "\n".join(documents)
    #return ""

chain_empty = (
    {"context": empty_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
)

chain_wiki = (
    {"context": wikipedia_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
)

# Extend the chain with the summarization step
chain_wiki_summarize = (
    {
        "context": wikipedia_retriever,  # Retrieve initial context
        "question": RunnablePassthrough(),  # Pass through the question
    }
    | summarize  # Apply the summarization step
    | model_small
)

chain_wiki_summarize_full = (
    {"context": chain_wiki_summarize.invoke, "question": RunnablePassthrough()}
    | prompt
    | model
)

'''
chain = (
    RunnablePassthrough.assign(question=question).assign(
        context=fake_retriever
    )
    | qa_prompt
    | llm
    | StrOutputParser()
)
'''

'\nchain = (\n    RunnablePassthrough.assign(question=question).assign(\n        context=fake_retriever\n    )\n    | qa_prompt\n    | llm\n    | StrOutputParser()\n)\n'

In [8]:
question = "i want to hear about some famous people"
response = chain_empty.invoke(question)
print("LLAMA: " + response.content)

i want to hear about some famous people
LLAMA: I'll give you a brief rundown of some famous individuals from various fields:

**History**

* Cleopatra (Ancient Egyptian Queen)
* Julius Caesar (Roman General and Statesman)
* Albert Einstein (Theoretical Physicist who developed the Theory of Relativity)

**Art and Entertainment**

* Leonardo da Vinci (Renaissance Artist and Inventor)
* William Shakespeare (English Playwright and Poet)
* Michael Jackson (American Singer, Songwriter, and Dancer)

**Science and Technology**

* Isaac Newton (Mathematician and Physicist who developed the Laws of Motion)
* Charles Darwin (Biologist who proposed the Theory of Evolution)
* Steve Jobs (Co-Founder of Apple Inc. and pioneer in Personal Computing)

**Politics and Leadership**

* Nelson Mandela (Anti-Apartheid Leader and President of South Africa)
* Mahatma Gandhi (Indian Independence Activist and Leader)
* Winston Churchill (Former Prime Minister of the United Kingdom during World War II)

Let me kn

In [9]:
question = "i want to hear about some famous people"
response = chain_wiki.invoke(question)
print("LLAMA: " + response.content)

LLAMA: Let's talk about some famous people mentioned in the text!

We have:

* Robert Greenfield, a former Rolling Stone magazine associate editor
* Philip Larkin, a British poet
* Joel Whitburn, a music historian
* David Foster Wallace, an American writer and tennis enthusiast (mentioned for his love of tennis, not just as a writer)
* Woody Allen, a film director and actor (mentioned for his movie Match Point)
* Luke Wilson, Jonathan Rhys Meyers, Scarlett Johansson, and Paul Bettany - all actors who starred in films related to tennis
* Queen Elizabeth II - the monarch who appointed the Beatles members of the Order of the British Empire
* Many members of the Beatles themselves: John Lennon, Paul McCartney, George Harrison, and Ringo Starr

These are just a few examples, but there are many other famous people mentioned in the text. Would you like to know more about any specific one?


In [18]:
question = "i want to hear about some famous people"
response = chain_wiki_summarize.invoke(question)
print("LLAMA: " + response.content)

LLAMA: Here is a summarized version of the text with only the requested information:

Robert Greenfield, British poet
Philip Larkin, British poet, described their work as enchanting
The Beatles have won numerous awards including 7 Grammy Awards and 15 Ivor Novello Awards
They are inducted into the Rock and Roll Hall of Fame (1988)
The Recording Industry Association of America certifies they sell over 183 million units in the US
They were collectively included in Time magazine's compilation of the 20th century's 100 most influential people


In [19]:
question = "i want to hear about some famous people"
response = chain_wiki_summarize_full.invoke(question)
print("LLAMA: " + response.content)

LLAMA: Here's a summary of the text that lists some famous people:

The text mentions several well-known individuals, including authors, actors, and musicians. These people are:

* David Foster Wallace, an American author and journalist
* William Shakespeare, a renowned English playwright and poet
* Luke Wilson, an American actor and musician
* Scarlett Johansson, a talented American actress
* Paul Bettany, a British actor
* Kirsten Dunst, a Danish-American actress
* Jonathan Rhys Meyers, an Irish actor
* Jeff Daniels, an American actor
* Laura Linney, an accomplished American actress

Would you like to know more about any of these individuals?
